### Monta Tabela de Anotação

In [11]:
import collections
import re
import pandas as pd

dictF = collections.OrderedDict()
anotador = 0
cont = 1
str1 = ""

for filename in ['Clear_07_3_ISO_Andressa_Bressane.xml','Clear_07_3_ISO_Eric_Eltz.xml', 'Clear_07_3_ISO_Keli_Schell.xml' ]:
    
    with open(filename,'r') as file:
        cont = 1
        for line in file:

            if line.find("EM-ID") >= 0 and line.find("/EM") >= 0:
                str1 = re.findall('(?<=>).*?(?=<)', line)[0] + " - " + str(cont)
                categoria = re.findall('(?<=CATEG=").*?(?=")', line)[0]
                #print(str1)
                if str1 in dictF:
                    dictF[str1][anotador] = categoria
                else:
                    dictF[str1] = ['','','']
                    dictF[str1][anotador] = categoria
            
            elif line.find("EM-ID") >= 0:
                str1 = re.findall('(?<=>).*?(?=$)', line)[0]
                str1 = str1.replace("\n", "")
                categoria = re.findall('(?<=CATEG=").*?(?=")', line)[0]
            
            elif line.find("/EM") >= 0:
                str1 = str1 + " " + re.findall('(?<=^).*?(?=<)', line)[0] + " - " + str(cont)
                str1 = str1.replace("\n", "")
                #print(str1)
                if str1 in dictF:
                    dictF[str1][anotador] = categoria
                else:
                    dictF[str1] = ['','','']
                    dictF[str1][anotador] = categoria
                str1 = ""
            
            else:
                str1 = str1 + " " + line
                
            cont += 1
        
    anotador += 1
    
#print(len(dictF))
    
kappa_pd = pd.DataFrame.from_dict(dictF,orient='index')
kappa_pd.sample(10)

,0,1,2
substrato - 3954,,,CONTEXTOgeologicoDeBACIA
substrato quartzítico - 285,sedimentaresSILICICLASTICAS,,
tectônico - 152,,,CONTEXTOgeologicoDeBACIA
neoproterozoicas - 342,era,,era
falhas direcionais - 176,outro,,CONTEXTOgeologicoDeBACIA
empurrão - 2350,,CONTEXTOgeologicoDeBACIA,
empurrão - 2565,,,CONTEXTOgeologicoDeBACIA
sedimentos glaciogênicos - 915,,,CONTEXTOgeologicoDeBACIA
dobras - 3588,,,CONTEXTOgeologicoDeBACIA
deformação - 560,,,CONTEXTOgeologicoDeBACIA


In [5]:
classes = set()

for x in kappa_pd[0].unique():
    classes.add(x.lower())
    
for x in kappa_pd[1].unique():
    classes.add(x.lower())
    
for x in kappa_pd[2].unique():
    classes.add(x.lower())

classes

{'',
 'baciasedimentar',
 'contextogeologicodebacia',
 'eon',
 'era',
 'idade',
 'outro',
 'periodo',
 'sedimentarescarbonaticas',
 'sedimentaresquimicas',
 'sedimentaressiliciclasticas',
 'unidadeestratigrafica'}

### Montagem da Tabela para o Kappa

In [6]:
w, h = len(classes), len(kappa_pd);
kappa_table = [[0 for x in range(w)] for y in range(h)] 

row_id = 0
for index, row in kappa_pd.iterrows():
    class_id = 0
    for clas in classes:
        if row[0].lower() == clas:
            kappa_table[row_id][class_id] += 1
        if row[1].lower() == clas:
            kappa_table[row_id][class_id] += 1
        if row[2].lower() == clas:
            kappa_table[row_id][class_id] += 1
        class_id += 1
    row_id += 1
            
kappa_table[:10]

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
 [0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
 [2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
 [1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
 [2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]]

In [7]:
from statsmodels.stats.inter_rater import fleiss_kappa
fleiss_kappa(kappa_table)

0.16027657450019131

### Calculo Kappa de Identificação

In [14]:
w, h = 2, len(kappa_pd);
kappa_id_table = [[0 for x in range(w)] for y in range(h)]

row_id = 0
for index, row in kappa_pd.iterrows():
    class_id = 0
    for clas in classes:
        if row[0].lower() == clas:
            kappa_id_table[row_id][class_id] += 1
        if row[1].lower() == clas:
            kappa_id_table[row_id][class_id] += 1
        if row[2].lower() == clas:
            kappa_id_table[row_id][class_id] += 1
        if class_id == 0:
            class_id += 1
    row_id += 1
    
kappa_id_table[:10]

[[0, 3],
 [0, 3],
 [1, 2],
 [2, 1],
 [2, 1],
 [0, 3],
 [1, 2],
 [2, 1],
 [2, 1],
 [1, 2]]

In [15]:
fleiss_kappa(kappa_id_table)

-0.10478857001688172

### Calculo Kappa da Classificação

In [16]:
kappa_class_table = pd.DataFrame(data=kappa_table)
kappa_class_table = kappa_class_table[kappa_class_table[0]==0]
kappa_class_table.drop(kappa_class_table.columns[0],inplace=True,axis=1)
kappa_class_table.shape

(72, 11)

In [17]:
fleiss_kappa(kappa_class_table.as_matrix())

0.8855730178350697